<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.simplefilter('ignore')

%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.utils import shuffle
from sklearn.model_selection import (
   
    RandomizedSearchCV,
    train_test_split
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.random.seed(1234)
from scipy import linalg as linalg


from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (StandardScaler)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from scipy import stats as st


In [2]:

try :
    insurance  = pd.read_csv('/datasets/insurance.csv')
except:
    insurance  = pd.read_csv('insurance.csv')
display(insurance.head(10))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [3]:
def df_inf(data): 
    display(data.head())
    display(data.describe(include='all').T)
    display(data.info())
    for column in data.columns:
        display()
        display(column)
        display(data[column].unique())
    display(data[data.duplicated()])
    display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm') )
df_inf(insurance)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

'Пол'

...

array([1, 0, 2, 4, 3, 5, 6])

'Страховые выплаты'

array([0, 1, 2, 3, 5, 4])

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


,0
Пол,0.000000
Возраст,0.000000
Зарплата,0.000000
Члены семьи,0.000000
Страховые выплаты,0.000000


In [4]:
insurance.head()
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
insurance.dtypes

Пол                    int64
Возраст              float64
Зарплата             float64
Члены семьи            int64
Страховые выплаты      int64
dtype: object

In [6]:
insurance['Возраст']=pd.to_numeric(insurance['Возраст'], downcast='integer')
insurance['Зарплата']=insurance['Зарплата'].astype('int')
insurance['Пол']=insurance['Пол'].astype('int8')
insurance['Члены семьи']=insurance['Члены семьи'].astype('int8')
insurance['Страховые выплаты']=insurance['Страховые выплаты'].astype('int8')

In [7]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int8 
 1   Возраст            5000 non-null   int8 
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int8 
 4   Страховые выплаты  5000 non-null   int8 
dtypes: int64(1), int8(4)
memory usage: 58.7 KB


In [8]:
insurance.dtypes

Пол                   int8
Возраст               int8
Зарплата             int64
Члены семьи           int8
Страховые выплаты     int8
dtype: object

Изменили типы данных для того чтобы снизить нагрузку на память

Вывод .

На первый взгляд данные были уже предобработаны. Пропусков и дубликатов нет. Соблюдается равенство по полу .  страховых выплат примерно 15%

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:**
пусть матрица умножилась на вектор $P$
тогда после умножения матрица стала матрицей $X1 , X1=XP$
вектор весов $ w = (X^T X)^{-1} X^Ty $

$A= XW$

$A1=X1W1$

$A1=X1*W1=(XP)W1$

$W1=((XP)^T(XP))^{-1}(XP)^Ty$

$W1=(P^TX^TXP)^{-1}P^TX^Ty$

$W1=P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty$

$W1=P^{-1}(X^TX)^{-1}EX^Ty$

$W1=P^{-1}(X^TX)^{-1}X^T$

$W1=P^{-1}W$

W1 не равен W 

Выразим  вектор  А1 через X1 и W1

$A1=X1W1$

$A1 = (XP)P^{-1}W = XEW = XW = A$ так как
$AE = EA = A$
A1=A поэтому качество модели не изменилось



Параметр w b w1 связаны $W1=P^{-1}W$

## Алгоритм преобразования

**Алгоритм**

1)Генерируем матрицу Z у которой число строк и число столбцов равнялось числу столбцов у изначальной матрицы Х - это нужно для того чтобы матрица Х не поменяла свою размерность

2) Проверяем матрицу Z  на обратимость, так как если она не будет иметь обратной матрицы то формула поиска коеффициента w1  работать не будет

3)Умножаем матрицу Х на матрицу Z

**Обоснование**

Матрица преобразования должна иметь размерность совпадающую числом строк с числом столбцов пребразуемой матрицы а также быть квадратной (одно из условий  для того чтобы у нее была обратная матрица). Также у этой матрицы должен быть определитель не равный нули иначе от нее невозможно будет найти обратную матрицу
 

## Проверка алгоритма

Создал функцию преобразования матрицы

In [9]:
def transform_matrix(matrix,low,high,):
    rows=matrix.shape[1]
    columns=rows
    matrix_2=np.random.randint (low, high, (rows, columns))
    while linalg.det(matrix_2) ==0:
        matrix_2=np.random.randint (low+1, high+1, (rows, columns))
    return matrix @ matrix_2
        

In [10]:
features=insurance[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']]
target=insurance['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=1234)

model_lr = make_pipeline(StandardScaler(),LinearRegression())
parameters_lr = {'linearregression__normalize':[False]}
random_search = RandomizedSearchCV(model_lr, 
                                   param_distributions=parameters_lr, 
                                   n_iter=1, 
                                   cv=3, 
                                   scoring = 'r2',
                                      ) 
random_search.fit(features_train, target_train) 
lr_rougher =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))


In [11]:
mod=make_pipeline(
                
                StandardScaler(),
                LinearRegression()
            )

In [12]:
model = LinearRegression()
model.fit(features_train, target_train)
R2 = r2_score(target_test, model.predict(features_test))
print(R2)

0.4249620550874137


In [13]:
model = mod
model.fit(features_train, target_train)
R2 = r2_score(target_test, model.predict(features_test))
print(R2)

0.424962055087415


Обучили линейную модель с маштабированием признаков и без, результат не изменился

In [14]:
features_transform=transform_matrix(features,1,10)
features_train_transform, features_test_transform, target_train, target_test = train_test_split(
    features_transform, target, test_size=0.25, random_state=1234)

In [15]:
model_transrormized=LinearRegression()

model_transrormized.fit(features_train_transform, target_train)
R2 = r2_score(target_test, model_transrormized.predict(features_test_transform))
print(R2)

0.4249620550873595


In [16]:
model_transrormized=mod

model_transrormized.fit(features_train_transform, target_train)
R2 = r2_score(target_test, model_transrormized.predict(features_test_transform))
print(R2)

0.4249620550874137


Обучили линейную модель с маштабированием  защищенных преобразованных признаков и без маштабирования, результат не изменился.

Сравнили модели с защищенными данными и нет. Качество модели не изменилось 

Вывод. В ходе работы были загружены и исследованы данные. Для оптимизации работы типы данных некоторых столбцов были изменены на более подходящие. Затем было теоретически иссследована возможность преобразования матрицы для защиты данных. Затем была разрабоатана фнкция преобразования матрицы ,включающая в себя проверку преобразующей матрицы на возможность нахождения обратной. После чего была обучена линейная модель на маштабированных и обычных признаках . эффективность модели не изменилась. Затем была обучена модель на преобразованных для защиты признаках. Эти признаки также были проверены со стандартизацией и без. Эффективность всех моделей была одинакова.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x] Проведено сравнение качества моделей до и после преобразования